# ISCE2 Stack Processing of NISAR-simulated UAVSAR products

In this notebook, we will demonstrate how to use the Stripmap Stack processor of ISCE2 to process the NISAR-simulated products made from UAVSAR data: https://uavsar.jpl.nasa.gov/science/documents/nisar-sample-products.html


The example scene we will use the the `snjoaq` flight line 14511:

https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=Snjoaq_14511_13129_010_130719_L090_CX_02#related

This contains 22 simulated UAVSAR SLC products.

# Initial environment setup

Since we'll be using the a recent branch of ISCE2 which (as of this writing) requires a manual setup, so it will be helpful to setup a conda environment.

Scott's preferences for setting up environments:

- I use miniconda to install packages into my environment (this is widely used for geospatial packages, due to annoying build dependencies)
- The first thing I install is [mamba](https://github.com/mamba-org/mamba), which makes it possible to install/ add new packages to an existing environment in seconds to minutes, as opposed to hours. This isn't as widely used, but I think it should be.


After getting set up with [miniconda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html), I would create my environment (called `uavsarenv` here) as follows:

(*Note*: commands that start with $ are terminal commands)

```
$ conda config --add channels conda-forge
$ conda create -n uavsarenv --yes python=3.8 pip mamba jupyter numpy
$ conda activate uavsarenv
```

With this environment created and activated, relaunch this notebook using the new environment:

```
$ jupyter notebook 
```

For future installs, since we installed `mamba`, we will use that `mamba install` install in place of `conda install` (for example):

```
$ mamba install --yes matplotlib # for plotting
```

## UAVSAR download tool

The UAVSAR query/download tool is located here https://github.com/scottstanie/uavsar/  within the "uaquery" package. This can be installed with 

```
pip install git+https://github.com/scottstanie/uavsar/
```


In [4]:
!pip install git+https://github.com/scottstanie/uavsar/

  Cloning https://github.com/scottstanie/uavsar/ to /tmp/pip-req-build-odl38ip5
  Running command git clone -q https://github.com/scottstanie/uavsar/ /tmp/pip-req-build-odl38ip5
  Resolved https://github.com/scottstanie/uavsar/ to commit 6a370c5c687f909d3dceb34397dd42b49ebb3790
  Created wheel for uavsar: filename=uavsar-0.1.0-py3-none-any.whl size=25899 sha256=5b31d3d6559c77085683d0086e9335c6f4b77204786fc469bb3ee940e362346d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ct7ztt7f/wheels/8c/16/55/266262b594793a605159a046c5087ab32c2ea291025313466e
Successfully built uavsar


This will create a `uaquery` command, which we can test to see if the installation worked:

In [5]:
!uaquery --help

usage: uaquery [-h] [--file-type FILE_TYPE] [--start-date START_DATE]
               [--end-date END_DATE]
               [--nisar-mode {129a,129b,138a,138b,143a,143b}]
               [--pol {hh,hv,vh,vv,hhhh,hvhv,vvvv,hhhv,hhvv,hvvv}]
               [--out-dir OUT_DIR] [--query-only] [--url-file URL_FILE]
               [--quiet]
               flight_line

positional arguments:
  flight_line           UAVSAR flight line

optional arguments:
  -h, --help            show this help message and exit
  --file-type FILE_TYPE
                        Extension of radar product (default=ann)
  --start-date START_DATE
                        Starting date for query (formats accepts: YYYYMMDD,
                        YYYY-MM-DD, yymmdd)
  --end-date END_DATE   Ending date for query (formats accepts: YYYYMMDD,
                        YYYY-MM-DD, yymmdd)
  --nisar-mode {129a,129b,138a,138b,143a,143b}
                        If searching for NISAR-simulated products, NISAR mode
                   

## Data download

First, we'll download all the HDF5 files corresponding to flight line 14511. 

We're going to use the NISAR Mode 129A, which is the 20 MHz mode (this will have smaller files than the 143A, which also is 20 MHz, since the "frequency B" for 129 is only 5 MHz).

In [8]:
import os
import subprocess

# CHANGE ME: pick a location for processing the data
WORKDIR = "/data6/scott/jpl/snjoaq_14511_129A/"
# Within the working directory, make a folder to hold the initial data download
DATADIR = WORKDIR + "data/"


def mkdir_p(path):
    """Emulates bash `mkdir -p`, in python style
    Used for output directory creation
    """
    import errno

    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
mkdir_p(DATADIR)
os.chdir(DATADIR)

First, we'll download all the "annotation" files to check that we find 22 NISAR-simulated products:

In [8]:
!uaquery 14511 --nisar-mode 129a --file-type ann

[08/18 14:09:06] [INFO cli.py] Arguments for search:
[08/18 14:09:06] [INFO cli.py] {'flight_line': 14511, 'file_type': 'ann', 'start_date': None, 'end_date': None, 'nisar_mode': '129a', 'pol': 'vv', 'out_dir': '.', 'query_only': False, 'url_file': 'uavsar_download_urls_{file_type}{pol}{nisar_mode}.txt', 'quiet': False, 'verbose': True}
[08/18 14:09:06] [INFO cli.py] Searching and downloading to .
[08/18 14:09:06] [INFO query_uavsar.py] url_file = uavsar_download_urls_ann129a.txt
[08/18 14:09:06] [INFO query_uavsar.py] Querying https://uavsar.jpl.nasa.gov/cgi-bin/data.pl?fname=searchUavsar&args=14511&searchText=14511&args=080101&startDate=080101&args=300101&endDate=300101&args=PolSAR&modeList=PolSAR&args=L-band%2Csimulated-nisar&bandList=L-band%2Csimulated-nisar&args=single-pol%2Cquad-pol&polList=single-pol%2Cquad-pol&args=14511&flownData=14511&args=14511&lineID=14511&args=simulated-nisar&simulatedNisar=simulated-nisar
[08/18 14:09:07] [INFO query_uavsar.py] https://uavsar.jpl.nasa.gov

Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloaduav.jpl.nasa.gov/Release2l/Snjoaq_14511_14086_011_140616_L090_CX_03/Snjoaq_14511_14086_011_140616_L090_CX_129A_03.ann [following]
--2021-08-18 14:09:23--  https://downloaduav.jpl.nasa.gov/Release2l/Snjoaq_14511_14086_011_140616_L090_CX_03/Snjoaq_14511_14086_011_140616_L090_CX_129A_03.ann
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uavsar.jpl.nasa.gov/Release2l/Snjoaq_14511_14086_011_140616_L090_CX_03/Snjoaq_14511_14086_011_140616_L090_CX_129A_03.ann [following]
--2021-08-18 14:09:23--  https://uavsar.jpl.nasa.gov/Release2l/Snjoaq_14511_14086_011_140616_L090_CX_03/Snjoaq_14511_14086_011_140616_L090_CX_129A_03.ann
Resolving uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)... 13

Connecting to uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30131 (29K) [text/plain]
Saving to: ‘Snjoaq_14511_15002_010_150107_L090_CX_129A_03.ann’

100%[======================================>] 30,131      --.-K/s   in 0.04s   

2021-08-18 14:09:25 (752 KB/s) - ‘Snjoaq_14511_15002_010_150107_L090_CX_129A_03.ann’ saved [30131/30131]

[08/18 14:09:25] [INFO query_uavsar.py] wget --no-clobber http://downloaduav.jpl.nasa.gov/Release2n/Snjoaq_14511_15017_012_150310_L090_CX_03/Snjoaq_14511_15017_012_150310_L090_CX_129A_03.ann
--2021-08-18 14:09:25--  http://downloaduav.jpl.nasa.gov/Release2n/Snjoaq_14511_15017_012_150310_L090_CX_03/Snjoaq_14511_15017_012_150310_L090_CX_129A_03.ann
Resolving downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved P

Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_17044_015_170403_L090_CX_02/Snjoaq_14511_17044_015_170403_L090_CX_129A_02.ann [following]
--2021-08-18 14:09:28--  https://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_17044_015_170403_L090_CX_02/Snjoaq_14511_17044_015_170403_L090_CX_129A_02.ann
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uavsar.jpl.nasa.gov/Release2u/Snjoaq_14511_17044_015_170403_L090_CX_02/Snjoaq_14511_17044_015_170403_L090_CX_129A_02.ann [following]
--2021-08-18 14:09:28--  https://uavsar.jpl.nasa.gov/Release2u/Snjoaq_14511_17044_015_170403_L090_CX_02/Snjoaq_14511_17044_015_170403_L090_CX_129A_02.ann
Resolving uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)... 13

Connecting to uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29743 (29K) [text/plain]
Saving to: ‘Snjoaq_14511_18002_010_180207_L090_CX_129A_02.ann’

100%[======================================>] 29,743      --.-K/s   in 0.04s   

2021-08-18 14:09:30 (723 KB/s) - ‘Snjoaq_14511_18002_010_180207_L090_CX_129A_02.ann’ saved [29743/29743]

[08/18 14:09:30] [INFO query_uavsar.py] wget --no-clobber http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_18034_014_180720_L090_CX_02/Snjoaq_14511_18034_014_180720_L090_CX_129A_02.ann
--2021-08-18 14:09:30--  http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_18034_014_180720_L090_CX_02/Snjoaq_14511_18034_014_180720_L090_CX_129A_02.ann
Resolving downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved P

In [9]:
!ls | wc
!ls | head -2

     24      24    1165
Snjoaq_14511_13129_010_130719_L090_CX_129A_03.ann
Snjoaq_14511_13165_012_131031_L090_CX_129A_03.ann


Now we'll kick off the download for all HDF5 files from this flight path (took about 20 minutes for me):

In [10]:
!uaquery 14511 --nisar-mode 129a --file-type h5

[08/18 14:09:58] [INFO cli.py] Arguments for search:
[08/18 14:09:58] [INFO cli.py] {'flight_line': 14511, 'file_type': 'h5', 'start_date': None, 'end_date': None, 'nisar_mode': '129a', 'pol': 'vv', 'out_dir': '.', 'query_only': False, 'url_file': 'uavsar_download_urls_{file_type}{pol}{nisar_mode}.txt', 'quiet': False, 'verbose': True}
[08/18 14:09:58] [INFO cli.py] Searching and downloading to .
[08/18 14:09:58] [INFO query_uavsar.py] url_file = uavsar_download_urls_h5129a.txt
[08/18 14:09:58] [INFO query_uavsar.py] Found existing uavsar_download_urls_h5129a.txt to read from.
[08/18 14:09:58] [INFO query_uavsar.py] wget --no-clobber http://downloaduav.jpl.nasa.gov/Release2f/Snjoaq_14511_13129_010_130719_L090_CX_03/Snjoaq_14511_13129_010_130719_L090_CX_129_03.h5
--2021-08-18 14:09:58--  http://downloaduav.jpl.nasa.gov/Release2f/Snjoaq_14511_13129_010_130719_L090_CX_03/Snjoaq_14511_13129_010_130719_L090_CX_129_03.h5
Resolving downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)... 137.78

Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloaduav.jpl.nasa.gov/Release2l/Snjoaq_14511_14068_012_140529_L090_CX_03/Snjoaq_14511_14068_012_140529_L090_CX_129_03.h5 [following]
--2021-08-18 14:13:06--  https://downloaduav.jpl.nasa.gov/Release2l/Snjoaq_14511_14068_012_140529_L090_CX_03/Snjoaq_14511_14068_012_140529_L090_CX_129_03.h5
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uavsar.jpl.nasa.gov/Release2l/Snjoaq_14511_14068_012_140529_L090_CX_03/Snjoaq_14511_14068_012_140529_L090_CX_129_03.h5 [following]
--2021-08-18 14:13:06--  https://uavsar.jpl.nasa.gov/Release2l/Snjoaq_14511_14068_012_140529_L090_CX_03/Snjoaq_14511_14068_012_140529_L090_CX_129_03.h5
Resolving uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)... 137.78.249

Resolving uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)... 137.78.249.121
Connecting to uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3546145131 (3.3G)
Saving to: ‘Snjoaq_14511_14166_010_141113_L090_CX_129_02.h5’

100%[====================================>] 3,546,145,131 77.8MB/s   in 45s    

2021-08-18 14:16:53 (76.0 MB/s) - ‘Snjoaq_14511_14166_010_141113_L090_CX_129_02.h5’ saved [3546145131/3546145131]

[08/18 14:16:53] [INFO query_uavsar.py] wget --no-clobber http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_15002_010_150107_L090_CX_03/Snjoaq_14511_15002_010_150107_L090_CX_129_03.h5
--2021-08-18 14:16:53--  http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_15002_010_150107_L090_CX_03/Snjoaq_14511_15002_010_150107_L090_CX_129_03.h5
Resolving downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80

HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloaduav.jpl.nasa.gov/Release2m/Snjoaq_14511_16104_011_161207_L090_CX_02/Snjoaq_14511_16104_011_161207_L090_CX_129_02.h5 [following]
--2021-08-18 14:19:58--  https://downloaduav.jpl.nasa.gov/Release2m/Snjoaq_14511_16104_011_161207_L090_CX_02/Snjoaq_14511_16104_011_161207_L090_CX_129_02.h5
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uavsar.jpl.nasa.gov/Release2m/Snjoaq_14511_16104_011_161207_L090_CX_02/Snjoaq_14511_16104_011_161207_L090_CX_129_02.h5 [following]
--2021-08-18 14:19:59--  https://uavsar.jpl.nasa.gov/Release2m/Snjoaq_14511_16104_011_161207_L090_CX_02/Snjoaq_14511_16104_011_161207_L090_CX_129_02.h5
Resolving uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)... 137.78.249.121
Connecting to uavsar.jpl.nasa.gov (uavsar.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP

HTTP request sent, awaiting response... 200 OK
Length: 3540593196 (3.3G)
Saving to: ‘Snjoaq_14511_17117_011_171026_L090_CX_129_02.h5’

100%[====================================>] 3,540,593,196 67.3MB/s   in 48s    

2021-08-18 14:23:54 (70.6 MB/s) - ‘Snjoaq_14511_17117_011_171026_L090_CX_129_02.h5’ saved [3540593196/3540593196]

[08/18 14:23:54] [INFO query_uavsar.py] wget --no-clobber http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_18002_010_180207_L090_CX_02/Snjoaq_14511_18002_010_180207_L090_CX_129_02.h5
--2021-08-18 14:23:54--  http://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_18002_010_180207_L090_CX_02/Snjoaq_14511_18002_010_180207_L090_CX_129_02.h5
Resolving downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav.jpl.nasa.gov (downloaduav.jpl.nasa.gov)|137.78.249.121|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloaduav.jpl.nasa.gov/Release2u/Snjoaq_14511_18002_010_180207_

In [10]:
# verify we downloaded all .h5 files
!ls *h5 | wc -l
!ls *ann | wc -l

22
22


## ISCE2 Setup


To set up the version of ISCE with the UAVSAR HDF5 support, we will use the fork of ISCE2 on my repo:

Additionally, I've had the most success manually installing ISCE with `cmake`. If you are more familiar with `scons` then that should work too (though I could not make it compile).
https://github.com/isce-framework/isce2#cmake-experimental



In [11]:
# to clone the repository somehwere besides "~/repos", change these first two lines
mkdir_p(os.path.expanduser("~/repos/"))
os.chdir(os.path.expanduser("~/repos/"))

In [12]:
!git clone https://github.com/scottstanie/isce2/

fatal: destination path 'isce2' already exists and is not an empty directory.


In [9]:
os.chdir(os.path.expanduser("~/repos/isce2"))

According to https://github.com/isce-framework/isce2#cmake-experimental we need to first install some dependencies. We'll do so using anaconda.

<div class="alert alert-info">
Since the UAVSAR products are HDF5 products, we have an extra dependency that we must install: h5py
</div>
(Also, the stripmap stack seems to import matplotlib, so we'll add that as well)

In [5]:
# Assuming that you have set up an Anaconda environment described above, and installed `mamba`
!mamba install --yes -c conda-forge gdal cython fftw cmake scipy matplotlib h5py


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.15.3) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['gdal', 'cython', 'fftw', 'cmake']

conda-forge/linux-64     Using cache
conda-forge/noarch       

Extracting   [>                                        ] (00m:00s)       1 / 49
Finished fftw                                 (00m:00s)               6 MB     18 MB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Finished json-c                               (00m:00s)             274 KB    710 KB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Finished giflib                               (00m:00s)              77 KB    199 KB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>    

Extracting   [>                                        ] (00m:00s)       1 / 49
Finished libwebp-base                         (00m:00s)             845 KB      1 MB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Finished pixman                               (00m:00s)             627 KB      1 MB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Finished boost-cpp                            (00m:00s)              16 MB     30 MB/s
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Extracting   [>                                        ] (00m:00s)       1 / 49
Finished librttopo 

Finished poppler                              (00m:00s)              16 MB     20 MB/s
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [=>                                       ] (00m:00s)       2 / 49
Finished cairo                                (00m:00s)               1 MB      2 MB/s
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [=>                                       ] (00m:00s)       2 / 49
Finished xorg-libxext                         (00m:00s)              54 KB     60 KB/s
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [=>                                       ] (00m:00s)       2 / 49
Extracting   [==>                                      ] (00m:00s)       3 / 49
Extracting   [==>  

Extracting   [==>                                      ] (00m:00s)       3 / 49
Extracting   [==>                                      ] (00m:00s)       3 / 49
Extracting   [==>                                      ] (00m:01s)       3 / 49
Finished libspatialite                        (00m:00s)               4 MB      4 MB/s
Extracting   [==>                                      ] (00m:01s)       3 / 49
Extracting   [==>                                      ] (00m:01s)       3 / 49
Finished libnetcdf                            (00m:00s)               1 MB      1 MB/s
Extracting   [==>                                      ] (00m:01s)       3 / 49
Extracting   [==>                                      ] (00m:01s)       3 / 49
Extracting   [==>                                      ] (00m:01s)       3 / 49
Extracting   [==>                                      ] (00m:01s)       3 / 49
Finished xorg-renderproto                     (00m:00s)               9 KB      7 KB/s
Extracting   [==>  

Extracting   [==========>                              ] (00m:02s)      13 / 49
Extracting   [==========>                              ] (00m:02s)      13 / 49
Extracting   [===========>                             ] (00m:02s)      14 / 49
Extracting   [===========>                             ] (00m:02s)      14 / 49
Extracting   [============>                            ] (00m:02s)      15 / 49
Extracting   [============>                            ] (00m:02s)      15 / 49
Extracting   [=============>                           ] (00m:02s)      16 / 49
Extracting   [=============>                           ] (00m:02s)      16 / 49
Extracting   [=============>                           ] (00m:02s)      17 / 49
Extracting   [=============>                           ] (00m:02s)      17 / 49
Extracting   [==============>                          ] (00m:02s)      18 / 49
Extracting   [==============>                          ] (00m:02s)      18 / 49
Extracting   [===============>          

Extracting   [================================>        ] (00m:10s)      39 / 49
Extracting   [================================>        ] (00m:10s)      39 / 49
Extracting   [=================================>       ] (00m:10s)      40 / 49
Extracting   [=================================>       ] (00m:10s)      40 / 49
Extracting   [==================================>      ] (00m:11s)      41 / 49
Extracting   [==================================>      ] (00m:11s)      41 / 49
Extracting   [==================================>      ] (00m:11s)      42 / 49
Extracting   [==================================>      ] (00m:11s)      42 / 49
Extracting   [===================================>     ] (00m:11s)      43 / 49
Extracting   [===================================>     ] (00m:11s)      43 / 49
Extracting   [====================================>    ] (00m:11s)      44 / 49
Extracting   [====================================>    ] (00m:11s)      44 / 49
Extracting   [==========================

In [13]:
mkdir_p(os.path.expanduser("~/repos/isce2/build"))
mkdir_p(os.path.expanduser("~/repos/isce2/install"))

os.chdir(os.path.expanduser("~/repos/isce2/build"))


In [16]:
# the "CMAKE_PREFIX_PATH" option is telling cmake to look in our anaconda directory for libraries
!cmake  -DCMAKE_INSTALL_PREFIX=../install -DCMAKE_PREFIX_PATH=$CONDA_PREFIX ..

-- The C compiler identification is GNU 7.3.1
-- The CXX compiler identification is GNU 7.3.1
-- The Fortran compiler identification is GNU 7.3.1
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /opt/rh/devtoolset-7/root/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /opt/rh/devtoolset-7/root/usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting Fortran compiler ABI info
-- Detecting Fortran compiler ABI info - done
-- Check for working Fortran compiler: /opt/rh/devtoolset-7/root/usr/bin/gfortran - skipped
-- Checking whether /opt/rh/devtoolset-7/root/usr/bin/gfortran supports Fortran 90
-- Checking whether /opt/rh/devtoolset-7/root/usr/bin/gfortran supports Fortran 90 - yes
-- The CUDA compiler identification is NVIDIA 

    runtime library [libgomp.so] in /opt/rh/devtoolset-7/root/usr/lib/gcc/x86_64-redhat-linux/7 may be hidden by files in:
      /home/scott/miniconda3/envs/mapping/lib

  Some of these libraries may not be found correctly.
Call Stack (most recent call first):
  /home/scott/miniconda3/envs/uavsarenv/share/cmake-3.21/Modules/FindPython.cmake:575 (__Python_add_library)
  CMakeLists.txt:33 (_Python_add_library)
  .cmake/UseCython.cmake:131 (Python_add_library)
  components/zerodop/GPUgeo2rdr/CMakeLists.txt:1 (cython_add_module)


CMake Warning at /home/scott/miniconda3/envs/uavsarenv/share/cmake-3.21/Modules/FindPython/Support.cmake:3315 (add_library):
  Cannot generate a safe runtime search path for target GPUresampslc because
  files in some directories may conflict with libraries in implicit
  directories:

    runtime library [libgomp.so] in /opt/rh/devtoolset-7/root/usr/lib/gcc/x86_64-redhat-linux/7 may be hidden by files in:
      /home/scott/miniconda3/envs/mapping/lib

  Some of t

In [17]:
!make install -j3

Scanning dependencies of target combinedLib
Scanning dependencies of target utilLib
[  0%] Building CXX object components/iscesys/StdOEL/src/CMakeFiles/stdoelLib.dir/ScreenWriter.cpp.o
[  1%] Building Fortran object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/linalg3/src/linalg3Module.F.o
[  1%] Building Fortran object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/orbit/src/orbitModule.F.o
[  1%] Building Fortran object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/poly1d/src/poly1dModule.F.o
[  1%] Building Fortran object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/poly2d/src/poly2dModule.F.o
[  1%] Building C object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/linalg3/src/linalg3.c.o
[  1%] Building C object components/isceobj/Util/CMakeFiles/combinedLib.dir/Library/orbit/src/orbit.c.o
[  2%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/akima_reg.F.o
[  2%] Building C object components

[ 13%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/lookvec.F.o
[ 13%] Building CXX object components/isceobj/LineAccessor/CMakeFiles/LineAccessorLib.dir/src/LineAccessor.cpp.o
[ 13%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/matmat.F.o
[ 13%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/matvec.F.o
[ 13%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/norm.F.o
[ 13%] Building CXX object components/isceobj/LineAccessor/CMakeFiles/LineAccessorLib.dir/src/LineAccessorF.cpp.o
[ 13%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/orrmread1.F.o
[ 14%] Building Fortran object components/isceobj/Util/src/CMakeFiles/utilLib.dir/polint.F.o
/home/scott/repos/isce2/components/isceobj/Util/src/polint.F:28:48:

           if(den.eq.0.)pause 'failure in polint'
                                                1
[ 14%] Building CXX object components/isceob

[ 24%] Linking CXX shared module fitoff.so
[ 24%] Building Fortran object components/mroipac/getPegInfo/CMakeFiles/get_peg_info.dir/src/get_peg_infoSetState.F.o
[ 24%] Building C object components/mroipac/grass/CMakeFiles/libgrass.dir/src/trees.c.o
[ 24%] Building Fortran object components/mroipac/getPegInfo/CMakeFiles/get_peg_info.dir/src/get_peg_infoAllocateDeallocate.F.o
[ 24%] Building C object components/mroipac/grass/CMakeFiles/libgrass.dir/src/residue.c.o
[ 24%] Built target fitoff
[ 24%] Building C object components/stdproc/alosreformat/CMakeFiles/alosLib.dir/ALOS_lib/src/cfft1d.c.o
[ 24%] Building Fortran object components/mroipac/getPegInfo/CMakeFiles/get_peg_info.dir/src/get_peg_infoGetState.F.o
[ 24%] Linking C shared library libgrass.so
[ 24%] Linking CXX shared module get_peg_info.so
[ 24%] Built target libgrass
Scanning dependencies of target formslcLib
[ 24%] Building C object components/stdproc/alosreformat/CMakeFiles/alosLib.dir/ALOS_lib/src/find_fft_length.c.o
[ 25%]

[ 34%] Building CXX object components/contrib/PyCuAmpcor/CMakeFiles/PyCuAmpcor.dir/PyCuAmpcor.cxx.o
In file included from /home/scott/miniconda3/envs/uavsarenv/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:0,
                 from /home/scott/miniconda3/envs/uavsarenv/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/scott/miniconda3/envs/uavsarenv/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/scott/repos/isce2/build/components/contrib/PyCuAmpcor/PyCuAmpcor.cxx:629:
/home/scott/miniconda3/envs/uavsarenv/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
/home/scott/repos/isce2/contrib/geo_autoRIFT/geogrid/src/geogrid.cpp: In member fun

[ 34%] Building CUDA object components/contrib/PyCuAmpcor/CMakeFiles/PyCuAmpcor.dir/src/GDALImage.cu.o
[ 34%] Linking CXX shared module geogrid.so
[ 34%] Linking CXX shared module autoriftcore.so
[ 34%] Built target geogrid
[ 34%] Building CUDA object components/contrib/PyCuAmpcor/CMakeFiles/PyCuAmpcor.dir/src/cuAmpcorChunk.cu.o
[ 34%] Built target autoriftcore
[ 34%] Building CXX object components/contrib/splitSpectrum/CMakeFiles/splitSpectrum.dir/splitSpectrum.cxx.o
[ 34%] Building CXX object components/contrib/splitSpectrum/CMakeFiles/splitSpectrum.dir/src/splitRangeSpectrum.cc.o
[ 34%] Linking CXX shared module splitSpectrum.so
[ 35%] Built target splitSpectrum
[ 36%] Building CUDA object components/contrib/PyCuAmpcor/CMakeFiles/PyCuAmpcor.dir/src/cuAmpcorController.cu.o
[ 36%] Building CUDA object components/contrib/PyCuAmpcor/CMakeFiles/PyCuAmpcor.dir/src/cuAmpcorParameter.cu.o
[ 36%] Building C object components/contrib/alos2filter/CMakeFiles/alos2filter.dir/src/psfilt1.c.o
/hom

...

-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/LineAccessor/LineAccessor.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/isceobj/LineAccessor/LineAccessor.so" to "/home/scott/repos/isce2/install/lib64"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/LineAccessor/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/LineAccessor/LineAccessorPy.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/Location/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/Location/Coordinate.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/Location/Offset.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/Location/Peg.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/isceobj/Location/SCH.py
-- Instal

-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/language/lexis/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/language/lexis/pragmatics.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/language/lexis/semantics.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/language/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/language/errors.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/reserved/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/reserved/glyphs.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/iscesys/Parsers/rdf/reserved/words.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/i

-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/grass/libgrass.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/mroipac/grass/libgrass.so" to "/home/scott/repos/isce2/install/lib64"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/grass/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/grass/grass.py
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/icu/icu.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/mroipac/icu/icu.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/icu/__init__.py
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/icu/Icu.py
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/mroipac/looks/l

-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUgeo2rdr/GPUgeo2rdr.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUgeo2rdr/GPUgeo2rdr.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUgeo2rdr/__init__.py
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUresampslc/GPUresampslc.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUresampslc/GPUresampslc.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUresampslc/__init__.py
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/zerodop/GPUtopozero/GPUtopozero.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages

-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/contrib/PyCuAmpcor/PyCuAmpcor.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/components/contrib/splitSpectrum/__init__.py
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/contrib/splitSpectrum/splitSpectrum.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/contrib/splitSpectrum/splitSpectrum.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Installing: /home/scott/repos/isce2/install/packages/isce2/components/contrib/alos2filter/libalos2filter.so
-- Set runtime path of "/home/scott/repos/isce2/install/packages/isce2/components/contrib/alos2filter/libalos2filter.so" to "/home/scott/repos/isce2/install/lib64:/home/scott/miniconda3/envs/mapping/lib"
-- Up-to-date: /home/scott/repos/isce2/install/packages/isce2/

With ISCE successfully installed, it is suggested to put some of the folders on your `$PATH` and `$PYTHON` path.

In [21]:
print(os.path.expanduser("~/repos/isce2/install"))
print(os.listdir(os.path.expanduser("~/repos/isce2/install")))

/home/scott/repos/isce2/install
['packages', 'bin']


I have a ~/.bashrc which has the following section:

```bash
# for manual ISCE install:
export ISCE_INSTALL_ROOT="/home/scott/repos/isce2/install/"
export ISCE_HOME=$ISCE_INSTALL_ROOT/packages/isce
export PYTHONPATH=$ISCE_INSTALL_ROOT/packages:$PYTHONPATH
export PATH=$ISCE_HOME/applications:$PATH

# Since we can only add 1 type of stack processor to the "$PATH" at a time...time...
isce_add_strip() {
    export PATH=$PATH:/home/scott/repos/isce2/contrib/stack/stripmapStack/
}
isce_add_tops() {
    export PATH=$PATH:/home/scott/repos/isce2/contrib/stack/topsStack/
}
# This lets us call from the terminal `isce_add_strip` to setup the stripmap functions
```

In [37]:
# Run this cell if you don't want to alter your ~/.bashrc file

# also remember to change the directory path to match where you git-cloned ISCE
ISCE_REPO = "/home/scott/repos/isce2"

ISCE_INSTALL_ROOT = ISCE_REPO + "/install/"
ISCE_HOME = ISCE_INSTALL_ROOT + "/packages/isce"

os.environ["ISCE_INSTALL_ROOT"] = ISCE_INSTALL_ROOT
os.environ["ISCE_HOME"] = ISCE_HOME
os.environ["PATH"] = "{}/applications:{}".format(ISCE_INSTALL_ROOT, os.environ["PATH"])
# "sys.path" is equivalent to the $PYTHONPATH variable
sys.path.insert(0, ISCE_INSTALL_ROOT + "/packages")

# Finally, make sure that the stripmapStack code is on the path as well
os.environ["PATH"] = "{}/contrib/stack/stripmapStack/:{}".format(ISCE_REPO, os.environ["PATH"])
#!export PATH=$PATH:/home/scott/repos/isce2/contrib/stack/stripmapStack/

If all has gone well, we should be able to import isce:

In [25]:
import isce
import isceobj

as well as the stripmap processing commands:

In [39]:
# If this fails, try the following to have the "stripmapStack" folder on your $PATH
# ISCE_REPO = "/home/scott/repos/isce2" # with your repo path
# os.environ["PATH"] = "{}/contrib/stack/stripmapStack/:{}".format(ISCE_REPO, os.environ["PATH"])
!stackStripMap.py -h

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
usage: stackStripMap.py [-h] -s SLCDIR [-x BBOX] [-w WORKDIR] -d DEM
                        [-m REFERENCEDATE] [-t DTTHR] [-b DBTHR] [-a ALKS]
                        [-r RLKS] [-S SENSOR] [-u UNWMETHOD] [-f FILTSTRENGTH]
                        [-L FL] [-H FH] [-B BANDWIDTH]
                        [--filter_sigma_x FILTERSIGMAX]
                        [--filter_sigma_y FILTERSIGMAY]
                        [--filter_size_x FILTERSIZEX]
                        [--filter_size_y FILTERSIZEY]
                        [--filter_kernel_rotation FILTERKERNELROTATION]
         

# Setting up the stripmap processor for the data

Let's go back to the folder where we downloaded data, and now we follow the steps for general stripmap stack processing https://github.com/scottstanie/isce2/tree/main/contrib/stack/stripmapStack#stripmap-stack-processor


In [42]:
os.chdir(WORKDIR)

In [43]:
!pwd

/data6/scott/jpl/snjoaq_14511_129A


*(These are now taken from the README in ISCE, repeated here for clarity)*

**2. Prepare DEM**

- a) create a folder for DEM;
- b) create a DEM using dem.py with SNWE of your study area in integer;
    - "south north west east" is the format of the bounding box, and it has to be in integer degree bounds
- c) Keep only ".dem.wgs84", ".dem.wgs84.vrt" and ".dem.wgs84.xml" and remove unnecessary files;
- d) fix the path of the file in the xml file of the DEM by using fixImageXml.py.
    - *Scott note*: rather than do this, I just create a symlink of the DEM in the root processing folder, shown below

In [45]:
mkdir_p(os.path.join(WORKDIR, "DEM"))
os.chdir(os.path.join(WORKDIR, "DEM"))

In [46]:
# We only care to process the southern part of the flight line, but the DEM will be much larger
!dem.py -a stitch -b 35 37 -121 -119 -r -s 1 -c
!rm demLat*.dem demLat*.dem.xml demLat*.dem.vrt
os.chdir(WORKDIR)
# This might be why the "fix xml file" step is included
ln -s DEM/demLat*.dem.wgs84* .

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O http://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N36W121.SRTMGL1.hgt.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   267  100   267    0     0   3172      0 --:--:-- --:--:-- --:--:--  3178
100 10.1M  100 10.1M    0     0  11.0M      0 --:--:-- --:--:-- --:--:-- 70.9M
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O http://e4ftl01.cr.usgs.gov/MEASURE

"Step 3" is to download the data. 

We have already downloaded the data, so we will skip that step here

**4. Prepare SAR data**
    
> Once all data have been downloaded, we need to unzip them and move them to different folders and getting ready for unpacking and then SLC generation. 

- This is the step that uses the recently added code for parsing the HDF5 products.

In [52]:
os.chdir(DATADIR)
SLCDIR = WORKDIR + "SLC/"
mkdir_p(SLCDIR)

In [57]:
# This step will take a few minutes: it will extract a binary SLC file from the HDF5 and organize by date.
# Also pulls some relevant platform metadata and stores in an ISCE "frame" object, in the form of a python "shelve"
!prepareUAVSAR_HDF5Stack.py --polarization VV --frequency A --input-dir . --output ../SLC

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
20130719
./Snjoaq_14511_13129_010_130719_L090_CX_129_03.h5
unpackFrame_UAVSAR_HDF5_SLC.py -i ./Snjoaq_14511_13129_010_130719_L090_CX_129_03.h5 -p VV -f A -o /data6/scott/jpl/snjoaq_14511_129A/SLC/20130719
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofw

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2021-08-18 16:09:03,729 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:09:03,730 - h5py._conv - DEBUG - Creating converter from 5 to 7
2021-08-18 16:09:03,730 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:09:03,730 - h5py._conv - DEBUG - Creating converter from 5 to 7
API open (R): /data6/scott/jpl/snjoaq_14511_129A/SLC/20140616/20140616.slc
API close:  /data6/scott/jpl/snjoaq_14511_129A/SLC/20140616/20140616.slc
/home/scott/repos/isce2/contrib/stack/stripmapStack/unpackFrame_UAVSAR_HDF5_SLC.py:70: RankWarning: Polyfit may be poor

API open (R): /data6/scott/jpl/snjoaq_14511_129A/SLC/20150310/20150310.slc
API close:  /data6/scott/jpl/snjoaq_14511_129A/SLC/20150310/20150310.slc
/home/scott/repos/isce2/contrib/stack/stripmapStack/unpackFrame_UAVSAR_HDF5_SLC.py:70: RankWarning: Polyfit may be poorly conditioned
  obj.extractDoppler()
gdal_translate -of ISCE HDF5:"./Snjoaq_14511_15017_012_150310_L090_CX_129_03.h5":"//science/LSAR/SLC/swaths/frequencyA/VV" /data6/scott/jpl/snjoaq_14511_129A/SLC/20150310/20150310.slc
Input file size is 2640, 31203
0...10...20...30...40...50...60...70...80...90...100 - done.
2640 31203
20150427
./Snjoaq_14511_15053_013_150427_L090_CX_129_03.h5
unpackFrame_UAVSAR_HDF5_SLC.py -i ./Snjoaq_14511_15053_013_150427_L090_CX_129_03.h5 -p VV -f A -o /data6/scott/jpl/snjoaq_14511_129A/SLC/20150427
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.

0...10...20...30...40...50...60...70...80...90...100 - done.
2640 31355
20170927
./Snjoaq_14511_17103_008_170927_L090_CX_129_02.h5
unpackFrame_UAVSAR_HDF5_SLC.py -i ./Snjoaq_14511_17103_008_170927_L090_CX_129_02.h5 -p VV -f A -o /data6/scott/jpl/snjoaq_14511_129A/SLC/20170927
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2021-08-18 16:12:21,794 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:12:21,794 - h5py._conv - DEBUG - Creating converter from 5 to 7
2021-08-18 16:12:21,794 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:12:21,794 - h5py._conv - DEBUG - Creating converter fr

2021-08-18 16:13:54,962 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:13:54,962 - h5py._conv - DEBUG - Creating converter from 5 to 7
2021-08-18 16:13:54,962 - h5py._conv - DEBUG - Creating converter from 7 to 5
2021-08-18 16:13:54,962 - h5py._conv - DEBUG - Creating converter from 5 to 7
API open (R): /data6/scott/jpl/snjoaq_14511_129A/SLC/20181108/20181108.slc
API close:  /data6/scott/jpl/snjoaq_14511_129A/SLC/20181108/20181108.slc
/home/scott/repos/isce2/contrib/stack/stripmapStack/unpackFrame_UAVSAR_HDF5_SLC.py:70: RankWarning: Polyfit may be poorly conditioned
  obj.extractDoppler()
gdal_translate -of ISCE HDF5:"./Snjoaq_14511_18082_007_181108_L090_CX_129_02.h5":"//science/LSAR/SLC/swaths/frequencyA/VV" /data6/scott/jpl/snjoaq_14511_129A/SLC/20181108/20181108.slc
Input file size is 2640, 31197
0...10...20...30...40...50...60...70...80...90...100 - done.
2640 31197


Now we'll use the `stackStripMap.py` script to create the set of `run_` bash files. We'll give it the following options:

- -s SLC :
- -a 5 -r 3: (5, 3) looks in (azimuth, range)
- -u snaphu: unwrap using snaphu
- --nofocus: a flag indicating that we're starting with pre-made SLC files, not raw data
- --workflow interferogram: adds one extra run file beyond coregistering the SLCs that will create interferograms
- --bbox 35.8 36.4 -120.4 -119.75:  zoom into to the bottom half of the flight path, near the strongest known subsidence

In [60]:
os.chdir(WORKDIR)
!stackStripMap.py -s SLC/ -d DEM/demLat*.dem.wgs84 -a 5 -r 3 -u snaphu --nofocus --workflow interferogram --bbox "35.8 36.4 -120.4 -119.75"

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
dirs =  ['/data6/scott/jpl/snjoaq_14511_129A/SLC/20130719', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20131031', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20140117', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20140402', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20140529', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20140616', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20140814', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20141006', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20141113', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20150107', '/data6/scott/jpl/snjoaq_14511_129A/SLC/20150310', '/data6/s

2021-08-18 16:15:41,303 - isce.mroipac.baseline - INFO - Sampling time 1
2021-08-18 16:15:41,313 - isce.mroipac.baseline - INFO - Sampling time 2
Baseline at top/bottom: -35.262062 3.812603
Orbit.calculateHeight: self.position =  [-2617154.5265081604, -4430042.8601533165, 3777489.9865386505]
Orbit.calculateHeight: ellipsoid.a, ellipsoid.e2 =  6378137.0 0.0066943799901
Orbit.calculateHeight: self.position =  [-2617154.5265081604, -4430042.8601533165, 3777489.9865386505]
Orbit.calculateHeight: ellipsoid.a, ellipsoid.e2 =  6378137.0 0.0066943799901
Using entire span of image for estimating baselines
2021-08-18 16:15:41,335 - isce.mroipac.baseline - INFO - Sampling time 0
2021-08-18 16:15:41,345 - isce.mroipac.baseline - INFO - Sampling time 1
2021-08-18 16:15:41,354 - isce.mroipac.baseline - INFO - Sampling time 2
Baseline at top/bottom: 2.343348 -2.590267
Orbit.calculateHeight: self.position =  [-2617154.5265081604, -4430042.8601533165, 3777489.9865386505]
Orbit.calculateHeight: ellipsoi

##################
SLCs with min degree connection of 21.0
20130719
20131031
20140117
20140402
20140529
20140616
20140814
20141006
20141113
20150107
20150310
20150427
20150516
20161207
20170403
20170410
20170927
20171026
20180207
20180720
20180814
20181108
##################
number of pairs:  231
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_01_crop
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_02_reference
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_03_focus_split
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_04_geo2rdr_coarseResamp
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_05_refineSecondaryTiming
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_06_invertMisreg
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_07_fineResamp
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_08_grid_baseline
writing  /data6/scott/jpl/snjoaq_14511_129A/run_files/run_09_igram


The result of this step will be a set of bash files:

In [61]:
!ls run_files

run_01_crop			 run_05_refineSecondaryTiming.job
run_01_crop.job			 run_06_invertMisreg
run_02_reference		 run_06_invertMisreg.job
run_02_reference.job		 run_07_fineResamp
run_03_focus_split		 run_07_fineResamp.job
run_03_focus_split.job		 run_08_grid_baseline
run_04_geo2rdr_coarseResamp	 run_08_grid_baseline.job
run_04_geo2rdr_coarseResamp.job  run_09_igram
run_05_refineSecondaryTiming	 run_09_igram.job


# Running the stripmap files

There are two options now:

1. Run each one of these individually and inspect the results, watching for errors
2. Run them all using the command 
```bash
for f in `find run_files/ ! -name "*.job"`do time bash $f >> fullrun.log 2>&1; done
```
and inspect the cumulative logfile after.

I like single commands, so I usually try 2 first. But in this notebook, we'll list out each step individually.

## Optional: Adjustment to the run files for large workstations

(skip this if you are running on a laptop, or if you don't care whether a step in 12 hours as opposed to 1 hour)

I have a large workstation with 72 cores and a lot of RAM. There are some steps of the workflow which look like the following, where a command is run once per interferogram:

In [65]:
# number of interferograms/commands
!wc -l run_files/run_05_refineSecondaryTiming
# The commands look like
!head -3 run_files/run_05_refineSecondaryTiming

231 run_files/run_05_refineSecondaryTiming
stripmapWrapper.py -c /data6/scott/jpl/snjoaq_14511_129A/configs/config_refineSecondaryTiming_20130719_20131031
stripmapWrapper.py -c /data6/scott/jpl/snjoaq_14511_129A/configs/config_refineSecondaryTiming_20130719_20140117
stripmapWrapper.py -c /data6/scott/jpl/snjoaq_14511_129A/configs/config_refineSecondaryTiming_20130719_20140402


This step, "refineSecondaryTiming", happens to take a long time and use only 1 core for most of the time.

I speed this up on my workstation by changing the runfile to the following:
```bash
#!/bin/bash
set -euo pipefail

MAX_PROCS=30
CONFIG_DIR=/data6/scott/jpl/snjoaq_14511_129A/configs/

find $CONFIG_DIR -name "config_refineSecondaryTiming*" -print0 | \
    xargs -0 -I {} --max-procs $MAX_PROCS stripmapWrapper.py -c {}

```

which just takes the same lines that are run one-by-one in the original, and runs 30 at a time using `xargs`.

In [66]:
new_run_file = """
#!/bin/bash
set -euo pipefail

MAX_PROCS=30
CONFIG_DIR=/data6/scott/jpl/snjoaq_14511_129A/configs/

find $CONFIG_DIR -name "config_refineSecondaryTiming*" -print0 | \
    xargs -0 -I {} --max-procs $MAX_PROCS stripmapWrapper.py -c {}
"""
# Save the original just in case
!mv run_files/run_05_refineSecondaryTiming original_run_05_refineSecondaryTiming
os.chdir(WORKDIR)
with open("run_files/run_05_refineSecondaryTiming", "w") as f:
    f.write(new_run_file)

In [67]:
!cat run_files/run_05_refineSecondaryTiming


#!/bin/bash
set -euo pipefail

MAX_PROCS=30
CONFIG_DIR=/data6/scott/jpl/snjoaq_14511_129A/configs/

find $CONFIG_DIR -name "config_refineSecondaryTiming*" -print0 |     xargs -0 -I {} --max-procs $MAX_PROCS stripmapWrapper.py -c {}


Now we're ready to run the steps:

In [71]:
!find run_files/ ! -name "*.job"

run_files/
run_files/run_01_crop
run_files/run_02_reference
run_files/run_03_focus_split
run_files/run_04_geo2rdr_coarseResamp
run_files/run_06_invertMisreg
run_files/run_07_fineResamp
run_files/run_08_grid_baseline
run_files/run_09_igram
run_files/run_05_refineSecondaryTiming


In [68]:
# To run all at once! 
# for f in `find run_files/ ! -name "*.job"`do time bash $f >> fullrun.log 2>&1; done

## Step individual steps

In [73]:
!time bash run_files/run_01_crop &> log_run_01_crop.txt


real	0m18.219s
user	1m7.111s
sys	1m47.379s


In [74]:
!time bash run_files/run_02_reference &> log_run_02_reference.txt


real	2m23.890s
user	162m25.497s
sys	1m52.454s


In [75]:
!time bash run_files/run_03_focus_split &> log_run_03_focus_split.txt


real	0m0.934s
user	0m0.780s
sys	0m0.154s


In [76]:
!time bash run_files/run_04_geo2rdr_coarseResamp &> log_run_04_geo2rdr_coarseResamp.txt


real	16m23.597s
user	952m2.418s
sys	10m59.854s


In [77]:
!time bash run_files/run_05_refineSecondaryTiming &> log_run_05_refineSecondaryTiming.txt


real	137m15.256s
user	3958m28.205s
sys	16m40.640s


In [78]:
!time bash run_files/run_06_invertMisreg &> log_run_06_invertMisreg.txt


real	0m0.450s
user	0m3.554s
sys	0m5.226s


In [79]:
!time bash run_files/run_07_fineResamp &> log_run_07_fineResamp.txt


real	3m39.695s
user	32m7.719s
sys	2m12.879s


In [80]:
!time bash run_files/run_08_grid_baseline &> log_run_08_grid_baseline.txt


real	136m34.676s
user	134m5.732s
sys	4m48.129s


In [81]:
!time bash run_files/run_09_igram &> log_run_09_igram.txt


real	582m10.751s
user	830m4.034s
sys	34m49.174s
